In [ ]:

# pos_label = set()

# i = 0
# pos_sample=pd.DataFrame()
# for drug, targets in dict_DTI.items():
#     for target in targets:
#         pair_name=[str(drug) + '_' + target]
#         pos_label.update(pair_name)
#         if i==0:
#             pos_sample=pd.DataFrame(pd.concat([matrix_drug.loc[drug],matrix_target.loc[target]]),columns=pair_name)
#         else:
#             spl=pd.DataFrame(pd.concat([matrix_drug.loc[drug],matrix_target.loc[target]]),columns=pair_name)
#             pos_sample=pos_sample.join(spl)
#         i+=1
# '--------------------------------------'
# pos_sample = pos_sample.T
# print(pos_sample.shape)
# print('# of positive samples: ' + str(len(pos_label)))
# n_positive = len(pos_label)
# n_negative = n_positive
# neg_label_total = set()
# for ind1 in matrix_target.index:
#     for ind2 in matrix_drug.index:
#         neg_label_total.update([str(ind2) + '_' + str(ind1)])
# print(len(neg_label_total))
# neg_label_total = neg_label_total.difference(pos_label)
# neg_label = np.random.choice(list(neg_label_total), size=n_negative, replace=False)
# '--------------------------------------'
# pos_label = set()

# for drug, targets in dict_DTI.items():
#     for target in targets:
#         pair_name = [str(drug) + '_' + target]
#         pos_label.update(pair_name)
# print(pos_label)
# '--------------------------------------'


# neg_sample=pd.DataFrame()
# j=0
# # dtype_drug=np.issubdtype(matrix_drug.index.dtype,np.number)
# # dtype_target=np.issubdtype(matrix_target.index.dtype,np.number)
# for neg in neg_label:
#     drug, target = neg.split('_',1)
#     if j==0:
#         neg_sample = pd.DataFrame(pd.concat([matrix_drug.loc[drug], matrix_target.loc[target]]),
#                                       columns=[neg])
#     else:
#         df_tmp = pd.DataFrame(pd.concat([matrix_drug.loc[drug], matrix_target.loc[target]]), columns=[neg])
#         neg_sample = neg_sample.join(df_tmp)
#     j += 1
# neg_sample = neg_sample.T
# print(neg_sample.shape)
# neg_sample.to_csv('neg_sample.txt',sep='\t')
    

In [ ]:
import functools
def doublewrap(function):
    """
    A decorator decorator, allowing to use the decorator to be used without
    parentheses if no arguments are provided. All arguments must be optional.
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator


@doublewrap
def define_scope(function, scope=None, *args, **kwargs):
    """
    A decorator for functions that define TensorFlow operations. The wrapped
    function will only be executed once. Subsequent calls to it will directly
    return the result so that operations are added to the graph only once.
    The operations added by the function live within a tf.variable_scope(). If
    this decorator is used with arguments, they will be forwarded to the
    variable scope. The scope name defaults to the name of the wrapped
    function.
    """
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)
    return decorator


class SoftmaxClassifier:
    def __init__(self,x,y,DTI):
        self.n_class = 2
        self.feature_size=DTI.train.data.shape[1]
        self.n_train=DTI.train.data.shape[0]
        self.n_test=DTI.test.data.shape[0]
        self.x=x
        self.y=y
        self.prediction
        self.optimize
        self.accuracy
        self.cost
        
    @define_scope(initializer=tf.contrib.slim.xavier_initializer())
    def prediction(self):
        x=self.x
        W = tf.Variable(tf.zeros([self.feature_size, self.n_class]), name='W')
        b = tf.Variable(tf.zeros([self.n_class]), name='b')
        return tf.nn.softmax(tf.matmul(x, W) + b)
    
    @define_scope
    def cost(self):
        return tf.reduce_mean(-tf.reduce_sum(y * tf.log(self.prediction+1e-12), reduction_indices=[1]))    
    
    @define_scope
    def optimize(self):
        return tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(self.cost)   
    @define_scope
    def accuracy(self):
        correct_prediction = tf.equal(tf.argmax(self.prediction, 1), tf.argmax(self.y, 1))
        return tf.reduce_mean(tf.cast(correct_prediction, dtype='float32'))

In [ ]:
config = tf.ConfigProto(log_device_placement=True)
lr = 0.01
n_class = 2
feature_size=DTI.train.data.shape[1]
n_train=DTI.train.data.shape[0]
n_test=DTI.test.data.shape[0]
batch_size = 1000
training_iteration = 100

avg_cost_list = []
epoch = int(training_iteration * batch_size / DTI.train.data.shape[0])

x = tf.placeholder(tf.float32, [None, feature_size])
W = tf.Variable(tf.zeros([feature_size, n_class]), name='W')
b = tf.Variable(tf.zeros([n_class]), name='b')
prediction= tf.nn.softmax(tf.matmul(x, W) + b)
y = tf.placeholder(tf.float32, [None, n_class])

cost=tf.reduce_mean(-tf.reduce_sum(y * tf.log(prediction+1e-12), reduction_indices=[1]))   
optimize= tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)   
# classifier=SoftmaxClassifier(x,y,DTI)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
class roc_auc(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.validation_data[0])
        self.aucs.append(roc_auc_score(self.validation_data[1], y_pred))
        if max(self.aucs) == self.aucs[-1]:
            model.save_weights("weights.roc_auc.hdf5")
        print(" - auc: %0.4f" % self.aucs[-1])
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return